# MNIST Results

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format

In [2]:
def extract_metrics_gpu(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
        if "GPU BEFORE EVALUATION:" in line: 
            gpu_u, gpu_mem = eval(line.split(":")[-1])
        
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, (ram_peak-ram_init) * 1024, cpu_u + cpu_k, gpu_u, gpu_mem)

In [3]:
def calculate_metrics_gpu(files): 
    metrics = dict()
    
    for file in files: 
        arch, method, i = file.split("_") 
        
        if method in metrics: 
            metrics[method].append(extract_metrics_gpu(file))
        else: 
            metrics[method] = [extract_metrics_gpu(file)] 
    
    return metrics

In [4]:
def split_orderings(metrics): 
    res = {}
    for m in metrics: 
        for i in range(3): 
            res[m + str(i + 1)] = metrics[m][i]
    return res

## 1. MLP + Task-IL 

In [5]:
%cd ~/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-17-21-14/task_gpu_fi

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-17-21-14/task_gpu_fi


In [6]:
logs = !ls

In [7]:
metrics = calculate_metrics_gpu(logs)
metrics = split_orderings(metrics)

In [8]:
df = pd.DataFrame(metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)'])

In [9]:
methods = ["nr", "lr", "gr", "lgr", "grd", "lgrd"]
order = [] 
for method in methods: 
    for i in range(1, 4): 
        order.append(method + str(i))
df.filter(order)

,nr1,nr2,nr3,lr1,lr2,lr3,gr1,gr2,gr3,lgr1,lgr2,lgr3,grd1,grd2,grd3,lgrd1,lgrd2,lgrd3
Accuracy (%),98.59,97.39,98.94,98.56,97.32,98.93,98.96,98.60,99.16,98.79,98.43,99.01,99.03,98.78,99.21,99.03,98.82,99.31
Training time (s),49.20,48.10,51.00,48.90,54.80,51.80,79.10,80.50,74.80,73.40,69.80,77.90,81.50,80.30,70.80,75.50,70.10,75.60
RAM Usage (MB),2.98,4.43,4.12,6.03,5.90,5.62,2.51,2.30,2.31,4.44,3.86,4.14,2.47,2.61,2.23,4.86,4.80,4.72
CPU Usage (s),78.13,78.49,80.65,78.01,85.68,81.43,109.15,110.75,105.00,104.40,101.66,108.10,112.21,110.60,100.50,105.48,99.80,105.71
GPU Usage (%),4.80,5.00,5.17,3.80,3.50,3.83,11.25,11.67,11.75,9.38,8.57,8.00,11.33,11.56,11.75,8.38,9.00,8.12
GPU Memory (MB),88.00,86.00,90.00,12.00,12.00,12.00,60.00,60.00,60.00,6.00,6.00,6.00,60.00,60.00,60.00,6.00,6.00,6.00


## 2. MLP + Class-IL 

In [10]:
%cd ~/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-17-22-01/class_gpu_fi/

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-17-22-01/class_gpu_fi


In [11]:
logs = !ls

In [12]:
metrics = calculate_metrics_gpu(logs)
metrics = split_orderings(metrics)

In [13]:
df = pd.DataFrame(metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)'])

In [14]:
methods = ["nr", "lr", "gr", "lgr", "grd", "lgrd"]
order = [] 
for method in methods: 
    for i in range(1, 4): 
        order.append(method + str(i))
df.filter(order)

,nr1,nr2,nr3,lr1,lr2,lr3,gr1,gr2,gr3,lgr1,lgr2,lgr3,grd1,grd2,grd3,lgrd1,lgrd2,lgrd3
Accuracy (%),94.65,94.14,94.50,94.45,94.48,94.56,93.92,92.75,93.74,94.27,94.44,94.21,94.00,92.15,93.49,94.68,94.50,94.17
Training time (s),104.90,108.40,99.30,101.90,107.40,102.00,146.40,137.80,146.10,137.00,133.30,137.00,138.90,147.00,143.10,133.30,134.20,134.90
RAM Usage (MB),3.98,3.10,3.64,4.46,5.82,3.76,2.39,4.63,4.40,4.59,3.93,4.71,2.53,2.00,4.85,4.49,3.33,3.88
CPU Usage (s),163.36,165.56,157.35,159.66,164.86,160.54,206.90,198.13,204.35,197.29,192.77,196.44,198.12,205.33,200.95,192.80,194.67,193.68
GPU Usage (%),6.36,5.36,6.20,4.55,4.55,5.09,13.27,13.14,12.00,9.14,9.29,8.93,12.57,12.33,12.67,9.57,9.57,10.50
GPU Memory (MB),138.00,144.00,142.00,20.00,18.00,18.00,60.00,60.00,60.00,6.00,6.00,6.00,60.00,60.00,60.00,6.00,6.00,6.00


## 3. CNN + Task-IL

In [15]:
%cd ~/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-17-17-18/task_gpu_fi

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-17-17-18/task_gpu_fi


In [16]:
logs = !ls

In [17]:
metrics = calculate_metrics_gpu(logs)
metrics = split_orderings(metrics)

In [18]:
df = pd.DataFrame(metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)'])

In [19]:
methods = ["nr", "lr", "gr", "lgr", "grd", "lgrd"]
order = [] 
for method in methods: 
    for i in range(1, 4): 
        order.append(method + str(i))
df.filter(order)

,nr1,nr2,nr3,lr1,lr2,lr3,gr1,gr2,gr3,lgr1,lgr2,lgr3,grd1,grd2,grd3,lgrd1,lgrd2,lgrd3
Accuracy (%),98.78,99.20,98.64,98.39,99.20,98.47,99.30,99.64,99.11,99.20,99.56,99.20,99.36,99.54,99.12,99.32,99.61,99.29
Training time (s),57.50,56.00,54.90,50.30,54.10,51.70,83.50,73.10,77.50,80.50,75.70,73.10,82.80,80.20,80.60,74.30,78.70,77.30
RAM Usage (MB),4.87,3.68,3.62,4.66,5.00,5.17,2.09,2.48,2.49,3.71,3.46,3.87,2.41,2.37,2.42,3.30,3.80,3.79
CPU Usage (s),86.54,85.64,84.37,80.07,83.06,80.63,112.77,105.79,107.65,109.33,106.17,102.66,111.62,110.04,110.66,103.41,107.53,106.65
GPU Usage (%),7.83,8.33,7.33,5.83,6.67,6.67,13.78,14.38,14.38,9.44,10.12,12.38,12.67,14.44,13.78,11.12,10.62,10.00
GPU Memory (MB),86.00,92.00,86.00,12.00,12.00,12.00,40.00,40.00,40.00,4.00,4.00,4.00,40.00,40.00,40.00,4.00,4.00,4.00


## 4. CNN + Class-IL

In [20]:
%cd ~/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-17-17-59/class_gpu_fi/

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/mnist/2021-05-17-17-59/class_gpu_fi


In [21]:
logs = !ls

In [22]:
metrics = calculate_metrics_gpu(logs)
metrics = split_orderings(metrics)

In [23]:
df = pd.DataFrame(metrics, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)'])

methods = ["nr", "lr", "gr", "lgr", "grd", "lgrd"]
order = [] 
for method in methods: 
    for i in range(1, 4): 
        order.append(method + str(i))
df.filter(order)

,nr1,nr2,nr3,lr1,lr2,lr3,gr1,gr2,gr3,lgr1,lgr2,lgr3,grd1,grd2,grd3,lgrd1,lgrd2,lgrd3
Accuracy (%),96.34,96.73,96.16,94.52,94.97,95.25,95.11,95.99,94.73,91.58,92.51,92.41,94.71,95.92,94.47,92.81,94.66,93.10
Training time (s),209.50,202.00,217.10,220.30,197.70,211.10,291.80,299.00,300.70,282.60,286.80,275.00,294.10,303.00,286.90,288.60,272.30,275.40
RAM Usage (MB),4.44,4.53,3.33,5.01,6.59,1.44,3.96,1.74,2.24,3.22,2.78,2.38,2.39,2.27,2.24,3.39,3.44,3.66
CPU Usage (s),326.12,318.69,333.86,337.13,313.91,328.82,407.97,414.22,415.76,400.78,405.08,392.45,410.68,417.72,401.75,404.67,387.31,395.56
GPU Usage (%),9.43,9.62,9.86,7.04,7.20,7.27,15.50,14.43,15.29,11.55,10.38,11.86,15.73,14.81,15.69,10.34,10.86,11.14
GPU Memory (MB),144.00,142.00,140.00,18.00,18.00,18.00,38.00,40.00,40.00,4.00,4.00,4.00,40.00,40.00,40.00,4.00,4.00,4.00
